In [15]:
# Project Title
## Predicting Climate Disaster Risk in Vietnam Using Historical Rainfall and Temperature Data

In [16]:
# Description
## This project aims by using  machine learning techniques to analyze historical climate data, identify patterns of extreme events, and predict periods of disaster risk to support timely disaster management and mitigation .

In [12]:
# IMPORT AND INSTALLATION

In [4]:
!pip install pandas numpy matplotlib seaborn



[notice] A new release of pip is available: 25.1.1 -> 25.2
[notice] To update, run: python.exe -m pip install --upgrade pip


In [20]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import joblib 
from sklearn.model_selection import train_test_split  
from sklearn.ensemble import RandomForestClassifier   
from sklearn.metrics import classification_report, confusion_matrix   
from sklearn.preprocessing import StandardScaler       
from sklearn.linear_model import LogisticRegression    


In [9]:
pip install numpy pandas scikit-learn seaborn matplotlib

   ---------------------------------------- 0.0/8.7 MB ? eta -:--:--
   -------- ------------------------------- 1.8/8.7 MB 9.2 MB/s eta 0:00:01
   ------------------ --------------------- 3.9/8.7 MB 9.4 MB/s eta 0:00:01
   ------------------------------ --------- 6.6/8.7 MB 10.2 MB/s eta 0:00:01
   ------------------------------------- -- 8.1/8.7 MB 9.4 MB/s eta 0:00:01
   ---------------------------------------- 8.7/8.7 MB 9.3 MB/s eta 0:00:00
   ---------------------------------------- 0.0/38.5 MB ? eta -:--:--
   -- ------------------------------------- 2.1/38.5 MB 10.4 MB/s eta 0:00:04
   ---- ----------------------------------- 4.5/38.5 MB 10.9 MB/s eta 0:00:04
   ------- -------------------------------- 7.1/38.5 MB 11.1 MB/s eta 0:00:03
   --------- ------------------------------ 9.4/38.5 MB 10.9 MB/s eta 0:00:03
   ----------- ---------------------------- 11.5/38.5 MB 10.7 MB/s eta 0:00:03
   -------------- ------------------------- 13.9/38.5 MB 10.9 MB/s eta 0:00:03
   -------


[notice] A new release of pip is available: 25.1.1 -> 25.2
[notice] To update, run: python.exe -m pip install --upgrade pip


In [14]:
# Loading Data Set 

In [13]:
df = pd.read_csv("vietnamclimate.csv")
print(df.head())

  Month  MinTemperature   MeanTemperature  MaxTemperature  Precipitation  \
0   Jan            15.86            19.77           23.72          33.59   
1   Feb            16.79            20.69           24.63          28.86   
2   Mar            16.74            22.18           27.68          42.81   
3   Apr            21.32            25.38           29.48          67.88   
4   May            23.07            26.96           30.90         176.59   

      Period  Level  
0  1901-1930      1  
1  1901-1930      2  
2  1901-1930      3  
3  1901-1930      4  
4  1901-1930      5  


In [18]:
 # Basic Info 


In [21]:
df.info()
df.describe()
df.isnull().sum()


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 48 entries, 0 to 47
Data columns (total 7 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   Month            48 non-null     object 
 1   MinTemperature   48 non-null     float64
 2   MeanTemperature  48 non-null     float64
 3   MaxTemperature   48 non-null     float64
 4   Precipitation    48 non-null     float64
 5   Period           48 non-null     object 
 6   Level            48 non-null     int64  
dtypes: float64(4), int64(1), object(2)
memory usage: 2.8+ KB


Month              0
MinTemperature     0
MeanTemperature    0
MaxTemperature     0
Precipitation      0
Period             0
Level              0
dtype: int64